In [1]:
import os
import pandas as pd
import numpy as np
from numba import jit

from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'jet'
%matplotlib inline
%load_ext line_profiler

/home/tronco/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/tronco/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# files = ["informe_Lagunillas_20170728235145.xls",  "informe_Lota_Urbana_20170729002437.xls",
# "informe_Lota_Rural_20170728235710.xls",  "informe_Meteorologia_20170729002107.xls"]
# names = ["lagunillas", "lota_u", "lota_r", "meteo"]

# for file,name in zip(files,names):
#     data = pd.read_html("../data/"+file,header=0)[0]
#     d = data["Velocidad Viento"]
#     datetime = pd.to_datetime(data["Fecha"] + " " + data["Hora"],format="%d-%m-%Y %H:%M")
#     d.index = datetime
#     d.to_csv("../data/"+name+".csv")

In [3]:
@jit
def getDataWindowed(data,inSize,outSize):
    biggest = np.max([inSize,outSize])
    
    matrixIn = np.zeros((len(data)-2*biggest, inSize))
    matrixOut = np.zeros((len(data)-2*biggest, outSize))
    for i in range(len(data)-2*biggest):
        matrixIn[i,:] = data[i:i+inSize]
        matrixOut[i,:] = data[i+inSize+1:i+inSize+outSize+1]
    return matrixIn,matrixOut

In [4]:
def createFolds(dataSize, k):
    vector = np.arange(dataSize)
    splitted = np.array_split(vector,k+1)
    
    folds = []
    
    test_set = []
    for i in range(k):
        test_set = np.hstack((test_set, splitted[i]))
        val_set = splitted[i+1]
        folds.append((test_set.astype('int'),val_set.astype('int')))
    return folds

In [5]:
data = pd.read_csv("../../data/meteo.csv",index_col=0,names=["datetime","windspeed"])["windspeed"]

In [6]:
X,y = getDataWindowed(data, 12,5)

In [7]:
class ESN(BaseEstimator,RegressorMixin):
    def __init__(self, n_reservoir = 1000, spectral_radius = 0.135, sparsity=0,
                 leaking_rate=0.3, regularization=1, random_state = None, activation = np.tanh):
        self.n_inputs = None
        self.n_outputs = None
        self.n_reservoir = n_reservoir
        self.spectral_radius = spectral_radius
        self.sparsity = sparsity
        self.leaking_rate = leaking_rate
        self.regularization = regularization
        self.last_state = None
        self.activation = activation
        if random_state:
            if type(random_state) is int:
                self.random_state=np.random.RandomState(random_state)
            elif type(random_state) is np.random.RandomState:
                self.random_state = random_state
        else:
            self.random_state = np.random.RandomState()
        self.ridge = Ridge(alpha=regularization)

    def get_params(self,deep=True):
        params =  {'n_reservoir':self.n_reservoir,'spectral_radius':self.spectral_radius,  'sparsity':self.sparsity,
                  'leaking_rate': self.leaking_rate, "regularization":self.regularization, "activation": self.activation}
        if self.n_inputs and self.n_outputs:
            params["n_inputs"] = self.n_inputs
            params["n_outputs"] = self.n_outputs
        if self.random_state:
            params["random_state"] = self.random_state
        return params

    @jit
    def fit(self,X,y):
        in_rows,self.n_inputs = X.shape
        if len(y.shape) > 1:
            out_rows, self.n_outputs = y.shape
        else:
            out_rows = len(y)
            self.n_outputs = 1
        initLen = int(0.01*in_rows)

        #Raise exception
        assert(in_rows == out_rows)

        #Input length
        N = in_rows

        #Creating input weights
        self.Win = (self.random_state.rand(self.n_reservoir,1+self.n_inputs)-0.5) * 1

        #Creating Reservoir weights
        self.W = self.random_state.rand(self.n_reservoir,self.n_reservoir)-0.5
        #Sparsity
        self.W[self.random_state.rand(*self.W.shape) < self.sparsity] = 0
        #Spectral radius
        self.W *= self.spectral_radius

        #Creating state matrix
        X_states = np.zeros((N-initLen,1+self.n_inputs+self.n_reservoir))

        #Last state
        self.last_state  = np.zeros(self.n_reservoir)

        #Collecting states
        for t in range(N):
            u = X[t]
            #Calculating new state
            self.last_state = (1-self.leaking_rate)*self.last_state  + self.leaking_rate*self.activation( np.dot( self.Win, np.hstack((1,u)) ) \
                                                                    + np.dot( self.W, self.last_state  ) )
            if t >= initLen:
                X_states[t-initLen,:] = np.hstack((1,u,self.last_state ))


        Y = y[initLen:]

        #Getting the output weights using least squares
        self.ridge.fit(X_states,Y)
        
        return self

    @jit
    def predict(self,X, cont=False):
        Y = np.empty((len(X),self.n_outputs))
        if not cont:
            last_state = np.zeros_like(self.last_state)
        else:
            last_state = self.last_state
        for t,u in enumerate(X):
                last_state = (1 - self.leaking_rate) * last_state + self.leaking_rate*self.activation( np.dot( self.Win, np.hstack((1,u)))+ \
                                                                     + np.dot( self.W, last_state  ) )
                y = self.ridge.predict( np.hstack((1,u,last_state)).reshape(1,-1))
                Y[t,:] = y

        if cont:
            self.last_state = last_state
        return Y

In [8]:
# generate the ESN reservoir
inSize = 12
outSize = 12
resSize = 3000
a = 0.3 # leaking rate

# load the data
trainLen = 2000
testLen = 2000
initLen = 100

data = np.loadtxt('MackeyGlass_t17.txt')
X, y= getDataWindowed(data,inSize,outSize)
X_train, y_train = (X[:trainLen], y[:trainLen])
X_test, y_test = (X[trainLen:trainLen+testLen], y[trainLen:trainLen+testLen])

In [9]:
y.shape

(9976, 12)

In [10]:
esn = ESN(random_state=42,n_reservoir=resSize,sparsity=0.5)

In [11]:
esn.get_params()

{'activation': <ufunc 'tanh'>,
 'leaking_rate': 0.3,
 'n_reservoir': 3000,
 'random_state': <mtrand.RandomState at 0x7f0e98825090>,
 'regularization': 1,
 'sparsity': 0.5,
 'spectral_radius': 0.135}